In [ ]:
import nltk
from nltk.corpus import stopwords 
import pandas as pd
import numpy as np
import string
import time
import spacy
import gensim
from gensim.models.phrases import Phrases, Phraser

In [ ]:
path = 'file_path'
df = pd.read_excel(path,encoding='utf8',keep_default_na=False)
docs = df['Verbatim'].fillna('').tolist()

In [ ]:
def TextPreprocessSpaCy(docs):

    nlp = spacy.load('en_core_web_sm',disable=['ner'])
    
    stop_words = stopwords.words('english')
    stop_add = ['-pron-']
    stop_remove = []
    stop = [x for x in stop_words if x not in stop_remove] + stop_add
     
    pos = ['SYM','NUM','NOUN','ADJ']
    
    tok_train = []
    tok_doc = []
    for doc in nlp.pipe(docs):
        tok_sent = []
        sentences = doc.sents
        for sent in sentences:
            tokens = [token.lemma_.lower() for token in sent if token.pos_ in pos]
            tokens = [i for i in tokens if i not in stop]
            tokens = [i for i in tokens if i not in string.punctuation]
            tok_train.append(tokens)
            tok_sent.extend(tokens)
        tok_doc.append(tok_sent)
    
    return tok_train,tok_doc

#Run preprocessing
tokens = TextPreprocessSpaCy(docs)

#Phrases Modelling
bigram_model = Phrases(tokens[0],min_count=5,threshold=1)
bigram_phraser = Phraser(bigram_model)
trigram_model = Phrases(bigram_phraser[tokens[0]],min_count=5,threshold=1)
trigram_phraser = Phraser(trigram_model)

tokens_trigram = list(trigram_phraser[bigram_phraser[tokens[1]]])

# Keyword/Phrases exploration
Use below script to extract keyword phrases containing specific keywords

In [ ]:
phrases = []

for i in tokens_trigram:
    j = [x for x in i if '_' in x and 'word1' in x or 'word2' in x or 'word3' in x]
    phrases.extend(j)

df = pd.DataFrame(phrases,columns=['keyword'])
df.keyword.value_counts().head(40)